In [ ]:
# default_exp data.question_answering

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# data.question_answering

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for question/answering tasks.

In [ ]:
#export
import ast
from functools import reduce

from fastcore.all import *
from fastai.data.block import DataBlock, CategoryBlock, ColReader, ColSplitter
from fastai.imports import *
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import (
    AutoModelForQuestionAnswering, logging,
    PretrainedConfig, PreTrainedTokenizerBase, PreTrainedModel
)

from blurr.utils import BLURR
from blurr.data.core import HF_BaseInput, HF_BeforeBatchTransform, first_blurr_tfm

logging.set_verbosity_error()

In [ ]:
#hide_input
import pdb

from fastai.data.core import DataLoader, DataLoaders, TfmdDL
from fastai.data.external import untar_data, URLs
from fastai.data.transforms import *
from fastcore.test import *
from nbverbose.showdoc import show_doc

from blurr.utils import print_versions
from blurr.data.core import HF_TextBlock

os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("What we're running with at the time this documentation was generated:")
print_versions('torch fastai transformers')

In [ ]:
#hide
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

## Question/Answering tokenization, batch transform, and DataBlock methods

Question/Answering tasks are models that require two text inputs (a context that includes the answer and the question).  The objective is to predict the start/end tokens of the answer in the context)

In [ ]:
path = Path('./')
squad_df = pd.read_csv(path/'squad_sample.csv'); len(squad_df)

We've provided a simple subset of a pre-processed SQUADv2 dataset below just for demonstration purposes. There is a lot that can be done to make this much better and more fully functional.  The idea here is just to show you how things can work for tasks beyond sequence classification. 

In [ ]:
squad_df.head(2)

In [ ]:
model_cls = AutoModelForQuestionAnswering

pretrained_model_name = 'roberta-base' #'xlm-mlm-ende-1024'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

In [ ]:
#export
def pre_process_squad(
    # A row in your pd.DataFrame
    row, 
    # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
    hf_arch:str,   
    # A Hugging Face tokenizer
    hf_tokenizer:PreTrainedTokenizerBase,  
    # The attribute in your dataset that contains the context (where the answer is included) (default: 'context')
    ctx_attr:str='context', 
    # The attribute in your dataset that contains the question being asked (default: 'question')
    qst_attr:str='question', 
    # The attribute in your dataset that contains the actual answer (default: 'answer_text')
    ans_attr:str='answer_text'
):
    context, qst, ans = row[ctx_attr], row[qst_attr], row[ans_attr]
    
    tok_kwargs = {}

    if(hf_tokenizer.padding_side == 'right'):
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(qst, context, **tok_kwargs))
    else:
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(context, qst, **tok_kwargs))
                                                                       
    tok_ans = hf_tokenizer.tokenize(str(row[ans_attr]), **tok_kwargs)
    
    start_idx, end_idx = 0,0
    for idx, tok in enumerate(tok_input):
        try:
            if (tok == tok_ans[0] and tok_input[idx:idx + len(tok_ans)] == tok_ans): 
                start_idx, end_idx = idx, idx + len(tok_ans)
                break
        except: pass
            
    row['tokenized_input'] = tok_input
    row['tokenized_input_len'] = len(tok_input)
    row['tok_answer_start'] = start_idx
    row['tok_answer_end'] = end_idx
    
    return row

In [ ]:
show_doc(pre_process_squad)

The `pre_process_squad` method is structured around how we've setup the squad DataFrame above.

In [ ]:
squad_df = squad_df.apply(partial(pre_process_squad, hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), axis=1)

In [ ]:
max_seq_len= 128

In [ ]:
squad_df = squad_df[(squad_df.tok_answer_end < max_seq_len) & (squad_df.is_impossible == False)]

In [ ]:
#hide
squad_df.head(2)

In [ ]:
vocab = dict(enumerate(range(max_seq_len)))

In [ ]:
#export
class HF_QuestionAnswerInput(HF_BaseInput): pass

We'll return a `HF_QuestionAnswerInput` from our custom `HF_BeforeBatchTransform` so that we can customize the show_batch/results methods for this task.

In [ ]:
#export
class HF_QABeforeBatchTransform(HF_BeforeBatchTransform):
    """Handles everything you need to assemble a mini-batch of inputs and targets, as well as 
    decode the dictionary produced as a byproduct of the tokenization process in the `encodes` method.
    """
    def __init__(
        self, 
        # The abbreviation/name of your Hugging Face transformer architecture (e.b., bert, bart, etc..)
        hf_arch:str,   
        # A specific configuration instance you want to use
        hf_config:PretrainedConfig,   
        # A Hugging Face tokenizer
        hf_tokenizer:PreTrainedTokenizerBase,  
        # A Hugging Face model
        hf_model:PreTrainedModel,      
        # To control the length of the padding/truncation. It can be an integer or None, 
        # in which case it will default to the maximum length the model can accept. If the model has no 
        # specific maximum input length, truncation/padding to max_length is deactivated.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        max_length:int=None,             
        # To control the `padding` applied to your `hf_tokenizer` during tokenization. If None, will default to 
        # `False` or `'do_not_pad'.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        padding:Union[bool, str]=True, 
        # To control `truncation` applied to your `hf_tokenizer` during tokenization. If None, will default to
        # `False` or `do_not_truncate`.
        # See [Everything you always wanted to know about padding and truncation](https://huggingface.co/transformers/preprocessing.html#everything-you-always-wanted-to-know-about-padding-and-truncation)
        truncation:Union[bool, str]=True, 
        # The `is_split_into_words` argument applied to your `hf_tokenizer` during tokenization. Set this to `True`
        # if your inputs are pre-tokenized (not numericalized)
        is_split_into_words:bool=False, 
        # Any other keyword arguments you want included when using your `hf_tokenizer` to tokenize your inputs
        tok_kwargs={}, 
        # Keyword arguments to apply to `HF_BeforeBatchTransform`
        **kwargs
    ):         
        super().__init__(hf_arch, hf_config, hf_tokenizer, hf_model,
                         max_length=max_length, padding=padding, truncation=truncation, 
                         is_split_into_words=is_split_into_words, tok_kwargs=tok_kwargs, **kwargs)
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        for s in samples:
            # cls_index: location of CLS token (used by xlnet and xlm); is a list.index(value) for pytorch tensor's
            s[0]['cls_index'] = (s[0]['input_ids'] == self.hf_tokenizer.cls_token_id).nonzero()[0]
            # p_mask: mask with 1 for token than cannot be in the answer, else 0 (used by xlnet and xlm)
            s[0]['p_mask'] = s[0]['special_tokens_mask']
            
        return samples

By overriding `HF_BeforeBatchTransform` we can add other inputs to each example for this particular task.

In [ ]:
before_batch_tfm = HF_QABeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                             max_length=max_seq_len, truncation='only_second', 
                                             tok_kwargs={ 'return_special_tokens_mask': True })

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput), 
    CategoryBlock(vocab=vocab),
    CategoryBlock(vocab=vocab)
)

dblock = DataBlock(blocks=blocks, 
                   get_x=lambda x: (x.question, x.context),
                   get_y=[ColReader('tok_answer_start'), ColReader('tok_answer_end')],
                   splitter=RandomSplitter(),
                   n_inp=1)

In [ ]:
dls = dblock.dataloaders(squad_df, bs=4)

In [ ]:
b = dls.one_batch(); len(b), len(b[0]), len(b[1]), len(b[2])

In [ ]:
b[0]['input_ids'].shape, b[0]['attention_mask'].shape, b[1].shape, b[2].shape

In [ ]:
#export
@typedispatch
def show_batch(
    # This typedispatched `show_batch` will be called for `HF_QuestionAnswerInput` typed inputs
    x:HF_QuestionAnswerInput, 
    # Your targets
    y,              
    # Your raw inputs/targets
    samples,        
    # Your `DataLoaders`. This is required so as to get at the Hugging Face objects for 
    # decoding them into something understandable
    dataloaders,    
    # Your `show_batch` context
    ctxs=None, 
    # The maximum number of items to show
    max_n=6, 
    # Any truncation your want applied to your decoded inputs
    trunc_at=None, 
    # Any other keyword arguments you want applied to `show_batch`
    **kwargs
):  
    # grab our tokenizer
    tfm = first_blurr_tfm(dataloaders)
    hf_tokenizer = tfm.hf_tokenizer
    
    res = L()
    for sample, input_ids, start, end in zip(samples, x, *y):
        txt = hf_tokenizer.decode(sample[0], skip_special_tokens=True)[:trunc_at]

        ans_toks = hf_tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=False)[start:end]
        res.append((txt, (start.item(),end.item()), hf_tokenizer.convert_tokens_to_string(ans_toks)))
                       
    display_df(pd.DataFrame(res, columns=['text', 'start/end', 'answer'])[:max_n])
    return ctxs

The `show_batch` method above allows us to create a more interpretable view of our question/answer data.

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

## Summary

This module includes all the low, mid, and high-level API bits for extractive Q&A tasks data preparation.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()